In [84]:
import json
from pathlib import Path

from FbxCommon import InitializeSdkObjects, LoadScene

def load_fbx(file_path):
    # Initialize the SDK manager and scene
    sdk_manager, scene = InitializeSdkObjects()

    # Load the FBX file
    success = LoadScene(sdk_manager, scene, file_path)
    if not success:
        print(f"Failed to load FBX file: {file_path}")
        return None

    print(f"Successfully loaded FBX file: {file_path}")
    return scene

# Example usage
file_path = "BP_Mixamo_New10_Scene_1_18_0.FBX"  # Replace with the actual path to your FBX file
scene = load_fbx(file_path)

Successfully loaded FBX file: BP_Mixamo_New10_Scene_1_18_0.FBX


In [85]:
from fbx import FbxCriteria, FbxAnimStack, FbxAnimLayer

KFCURVENODE_T_X = "X"
KFCURVENODE_T_Y = "Y"
KFCURVENODE_T_Z = "Z"

KFCURVENODE_R_X = "X"
KFCURVENODE_R_Y = "Y"
KFCURVENODE_R_Z = "Z"
KFCURVENODE_R_W = "W"

KFCURVENODE_S_X = "X"
KFCURVENODE_S_Y = "Y"
KFCURVENODE_S_Z = "Z"


def getCurveKeys(pCurve):
    if pCurve:
        keyframes_values = []
        lKeyCount = pCurve.KeyGetCount()

        for lCount in range(lKeyCount):
            lKeyValue = pCurve.KeyGetValue(lCount)
            keyframes_values.append(float(lKeyValue))
    else:
        keyframes_values = None
    return keyframes_values


motion_data = {}

anim_stack = scene.GetSrcObject(FbxCriteria.ObjectType(FbxAnimStack.ClassId), 0)
anim_layer = anim_stack.GetSrcObject(FbxCriteria.ObjectType(FbxAnimLayer.ClassId), 0)

root_node = scene.GetRootNode().GetChild(0)

motion_data = {}
joints_names = []
def append_node_data(node):
    node_name = node.GetName()
    joints_names.append(node_name)

    # Register animation data
    motion_data[node_name + '_TX'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_X))
    motion_data[node_name + '_TY'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_Y))
    motion_data[node_name + '_TZ'] = getCurveKeys(node.LclTranslation.GetCurve(anim_layer, KFCURVENODE_T_Z))
    motion_data[node_name + '_RX'] = getCurveKeys(node.LclRotation.GetCurve(anim_layer, KFCURVENODE_R_X))
    motion_data[node_name + '_RY'] = getCurveKeys(node.LclRotation.GetCurve(anim_layer, KFCURVENODE_R_Y))
    motion_data[node_name + '_RZ'] = getCurveKeys(node.LclRotation.GetCurve(anim_layer, KFCURVENODE_R_Z))


   # call recursively for children
    child_count = node.GetChildCount()
    if child_count > 0:
        # call recursively
        for child_idx in range(child_count):
            child_node = node.GetChild(child_idx)
            append_node_data(child_node)
    return motion_data

motion_data = append_node_data(root_node)

In [86]:
def get_skeleton_hierarchy(root_node):
    """
    Returns the hierarchy of the skeleton as a list of indices,
    where each index represents the parent bone index.

    Parameters:
    - root_node: The root node of the skeleton hierarchy.

    Returns:
    - list: An array-like structure representing the parent indices of each bone.
    """
    hierarchy = []
    node_index_map = {}  # Maps node unique IDs to their indices

    def traverse(node, parent_index):
        # Assign an index to the current node
        current_index = len(hierarchy)
        node_id = node.GetUniqueID()  # Use unique ID as the key
        node_index_map[node_id] = current_index

        # Add the parent index to the hierarchy
        hierarchy.append(parent_index)

        # Traverse the children
        for i in range(node.GetChildCount()):
            child_node = node.GetChild(i)
            traverse(child_node, current_index)

    # Start traversal from the root node
    traverse(root_node, -1)

    return hierarchy


In [87]:
from fbx import FbxNodeAttribute, FbxAMatrix, FbxPose

def get_skeletal_information(scene):
    """
    Extracts skeletal information from the FBX scene, including the hierarchy
    and offsets (bind pose matrices).

    Parameters:
    - scene: The FbxScene object.

    Returns:
    - skeleton_data: A dictionary containing skeletal hierarchy and offset matrices.
    """
    skeleton_data = {
        "hierarchy": [],
        "offsets": {}
    }

    def find_bind_pose(node):
        """
        Finds the bind pose (offset matrix) for a given node from the scene.
        """
        bind_pose_matrix = FbxAMatrix()

        # Traverse poses in the scene to find the bind pose
        for i in range(scene.GetPoseCount()):
            pose = scene.GetPose(i)
            if pose.IsBindPose():
                for j in range(pose.GetCount()):
                    if pose.GetNode(j) == node:
                        bind_pose_matrix = pose.GetMatrix(j)
                        return bind_pose_matrix
        return bind_pose_matrix

    def matrix_to_offset(matrix):
        """
        Converts an FbxAMatrix to a list of three values representing the translation component.
        """
        translation = matrix.GetT()
        return [translation[0], translation[1], translation[2]]

    def traverse_skeleton(node, parent_index=-1):
        """
        Recursively traverses the skeleton hierarchy to gather data.
        """
        node_name = node.GetName()

        # Check if the node is part of the skeleton
        if node.GetNodeAttribute() and node.GetNodeAttribute().GetAttributeType() == FbxNodeAttribute.eSkeleton:
            # Add the node to the hierarchy
            current_index = len(skeleton_data["hierarchy"])
            skeleton_data["hierarchy"].append(parent_index)

            # Get the bind pose matrix
            bind_pose_matrix = find_bind_pose(node)
            skeleton_data["offsets"][node_name] = matrix_to_offset(bind_pose_matrix)

            # Traverse children
            for i in range(node.GetChildCount()):
                child_node = node.GetChild(i)
                traverse_skeleton(child_node, current_index)

    # Start traversal from the root node
    root_node = scene.GetRootNode()
    for i in range(root_node.GetChildCount()):
        traverse_skeleton(root_node.GetChild(i))

    return skeleton_data

In [88]:
skeleton_info = get_skeletal_information(scene)
skeleton_info.keys()

AttributeError: 'FbxMatrix' object has no attribute 'GetT'

In [89]:
data = {'motion_data': motion_data,
        'joints_names': joints_names,
        'parents': get_skeleton_hierarchy(root_node),
        # 'hierarchy': skeleton_info['hierarchy'],
        # 'offsets': skeleton_info['offsets'],
        }

with open(Path(file_path).stem + '.json', 'w') as fp:
    json.dump(data, fp, indent=2)